# 🧠 InsightSpike-AI Google Colab Demo

**Brain-Inspired Multi-Agent Architecture for Insight Detection**

このノートブックは、InsightSpike-AIをGoogle Colab環境で実行するためのデモです。

⚡ **GPU Runtime推奨**: Runtime > Change runtime type > GPU

## 🔧 セットアップ

In [ ]:
# リポジトリのクローン
!git clone https://github.com/miyauchikazuyoshi/InsightSpike-AI.git
%cd InsightSpike-AI

In [ ]:
# 環境セットアップ（GPU対応）
# 必要なスクリプトファイルに実行権限を付与
!chmod +x scripts/colab/setup_colab.sh
!chmod +x scripts/setup/setup.sh
!chmod +x scripts/setup/refactor_prepare.sh

# Pythonスクリプトにも実行権限を付与（必要に応じて）
!find scripts/ -name "*.py" -exec chmod +x {} \;

# Colabセットアップスクリプト実行
!./scripts/colab/setup_colab.sh

In [ ]:
# 包括的環境検証の実行
# setup_colab.shで自動実行されますが、個別でも実行可能
!python scripts/colab/test_colab_env.py

In [ ]:
# Faiss-GPU動作確認（詳細版）
import faiss
import numpy as np

print(f"🔍 Faiss Version: {faiss.__version__}")
print(f"🔍 GPU Resources Available: {hasattr(faiss, 'StandardGpuResources')}")

# GPU acceleration test
if hasattr(faiss, 'StandardGpuResources'):
    try:
        # Create test vectors
        dimension = 256
        n_vectors = 10000
        vectors = np.random.random((n_vectors, dimension)).astype('float32')
        
        # CPU Index
        index_cpu = faiss.IndexFlatL2(dimension)
        index_cpu.add(vectors)
        print(f"✅ CPU Index: {index_cpu.ntotal} vectors")
        
        # GPU Index
        gpu_res = faiss.StandardGpuResources()
        gpu_index = faiss.index_cpu_to_gpu(gpu_res, 0, index_cpu)
        print(f"✅ GPU Index: {gpu_index.ntotal} vectors")
        
        # Performance comparison
        import time
        query = np.random.random((1, dimension)).astype('float32')
        
        # CPU search
        start = time.time()
        cpu_distances, cpu_indices = index_cpu.search(query, 10)
        cpu_time = time.time() - start
        
        # GPU search
        start = time.time()
        gpu_distances, gpu_indices = gpu_index.search(query, 10)
        gpu_time = time.time() - start
        
        print(f"🚀 CPU Search Time: {cpu_time:.4f}s")
        print(f"🚀 GPU Search Time: {gpu_time:.4f}s")
        print(f"🚀 Speedup: {cpu_time/gpu_time:.2f}x")
        
    except Exception as e:
        print(f"❌ GPU Test Failed: {e}")
else:
    print("❌ GPU resources not available")

## 📊 データ準備

In [ ]:
# CLIコマンドアクセス確認
print("\n💻 CLI Command Access Validation:")

# Poetry CLIアクセステスト
try:
    import subprocess
    result = subprocess.run(['poetry', 'run', 'python', '-c', 'import insightspike.cli; print("CLI module accessible")'], 
                          capture_output=True, text=True, cwd='.')
    if result.returncode == 0:
        print("✅ Poetry CLI access: Available")
        print(f"   Output: {result.stdout.strip()}")
    else:
        print(f"❌ Poetry CLI access failed: {result.stderr}")
except Exception as e:
    print(f"❌ Poetry CLI test error: {e}")

# 直接アクセステスト
try:
    import sys
    import os
    old_path = sys.path.copy()
    if 'src' not in sys.path:
        sys.path.insert(0, 'src')
    
    import insightspike.cli
    print("✅ Direct CLI module access: Available")
    
    # PYTHONPATHテスト
    os.environ['PYTHONPATH'] = 'src'
    result = subprocess.run(['python', '-c', 'import insightspike.cli; print("Direct access works")'], 
                          capture_output=True, text=True, env=os.environ)
    if result.returncode == 0:
        print("✅ PYTHONPATH direct access: Available")
    else:
        print(f"⚠️ PYTHONPATH access issue: {result.stderr}")
    
    sys.path = old_path
except Exception as e:
    print(f"❌ Direct CLI access error: {e}")

print("\n🚀 Recommended CLI usage:")
print("   🎆 Primary:   !PYTHONPATH=src poetry run python -m insightspike.cli [command]")
print("   🔄 Fallback:  !PYTHONPATH=src python -m insightspike.cli [command]")

In [ ]:
# データの準備（サンプルデータ作成）
# 必要なディレクトリを作成
!mkdir -p data/raw data/processed data/embedding data/models

# サンプルデータファイルを作成
with open('data/raw/test_sentences.txt', 'w', encoding='utf-8') as f:
    f.write("""The aurora borealis is caused by charged particles from the sun interacting with Earth's magnetic field.
Quantum entanglement is a phenomenon where particles become correlated in ways that defy classical physics.
Artificial intelligence uses machine learning algorithms to process data and make predictions.
The human brain contains billions of neurons that communicate through synapses.
Photosynthesis converts sunlight into chemical energy in plants.
DNA contains the genetic instructions for all living organisms.
Gravity is a fundamental force that attracts objects with mass toward each other.
Evolution explains how species change over time through natural selection.
""")

print("✅ Data directories created")
print("✅ Sample data created in data/raw/test_sentences.txt")

In [ ]:
# エピソード記憶の構築
# Method 1: Poetry run (推奨)
!PYTHONPATH=src poetry run python -m insightspike.cli embed --path data/raw/test_sentences.txt

# Method 2: 直接実行 (Poetry問題時の代替)
# !PYTHONPATH=src python -m insightspike.cli embed --path data/raw/test_sentences.txt

# Method 3: Makefileを使用
# !make embed

In [ ]:
# 類似性グラフの構築
# Method 1: Poetry run (推奨)
!PYTHONPATH=src poetry run python -m insightspike.cli graph

# Method 2: 直接実行 (Poetry問題時の代替)
# !PYTHONPATH=src python -m insightspike.cli graph

## 🚀 デモ実行

In [ ]:
# 簡単なテスト実行
# Method 1: Poetry run (推奨)
!PYTHONPATH=src poetry run python -m insightspike.cli loop "What is quantum entanglement?"

# Method 2: 直接実行 (Poetry問題時の代替)
# !PYTHONPATH=src python -m insightspike.cli loop "What is quantum entanglement?"

In [ ]:
# フルPoC実行（可視化付き）
!PYTHONPATH=src poetry run python scripts/run_poc_simple.py "How does artificial intelligence work?"

## 🔍 インタラクティブテスト

In [ ]:
# Python環境でのインタラクティブテスト
import sys
import os
sys.path.append('src')

# 環境変数設定
os.environ['PYTHONPATH'] = 'src'

from insightspike.core.layers.layer2_memory_manager import L2MemoryManager
from insightspike.core.agents.main_agent import MainAgent

# メモリとエージェントの初期化
try:
    agent = MainAgent()
    if agent.initialize():
        print("✅ Agent initialized successfully")
        memory_stats = agent.l2_memory.get_memory_stats()
        print(f"✅ Memory loaded: {memory_stats.get('total_episodes', 0)} episodes")
    else:
        print("❌ Agent initialization failed")
except Exception as e:
    print(f"❌ Setup failed: {e}")

# 質問例
question = "What causes the aurora borealis?"
print(f"Question: {question}")

# 質問処理実行
try:
    result = agent.process_question(question, verbose=True)
    print(f"\n📊 Results:")
    print(f"Response: {result.get('response', 'No response')}")
    print(f"Reasoning Quality: {result.get('reasoning_quality', 0):.3f}")
    print(f"Spike Detected: {result.get('spike_detected', False)}")
    print(f"Cycles: {result.get('total_cycles', 0)}")
except Exception as e:
    print(f"❌ Question processing failed: {e}")

## 🔧 トラブルシューティング

### よくあるエラーと解決方法

1. **CUDA not available**
   - Runtime > Change runtime type > GPU を確認

2. **ModuleNotFoundError**
   - セットアップスクリプトを再実行
   - `!pip install [missing_package]` で個別インストール

3. **FileNotFoundError: data files**
   - データ準備セクションを再実行
   - `mkdir -p data/raw` でディレクトリ作成

4. **Memory issues**
   - ランタイムを再起動: Runtime > Restart runtime

5. **Import errors**
   - `sys.path.append('src')` が正しく設定されているか確認
   - PYTHONPATH 環境変数の設定を確認

6. **CLIコマンドエラー**
   - **Poetry CLIが動作しない場合**:
     ```python
     # 代替方法: 直接実行
     !PYTHONPATH=src python -m insightspike.cli [command]
     ```
   - **insightspike module not found**:
     ```python
     # 再インストール
     !pip install -e .
     # または
     !poetry install --no-deps
     ```
   - **CLIコマンド一覧確認**:
     ```python
     !PYTHONPATH=src python -m insightspike.cli --help
     ```

### 🚑 緊急時の代替方法

**Poetryが完全に動作しない場合**:
```python
# 1. 直接Pythonモジュールとして実行
import sys, os
sys.path.append('src')
os.environ['PYTHONPATH'] = 'src'

# 2. スクリプト直接実行
!PYTHONPATH=src python scripts/run_poc_simple.py "Your question"

# 3. ライブラリ直接使用
from insightspike.core.agents.main_agent import MainAgent
agent = MainAgent()
```